In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 Nano model
model = YOLO('yolov8n.pt')

# Path to your image
image_path = "data\State-wise_OLX\AN\AN1.jpg"  # Replace with your image path

# Load the image using OpenCV
image = cv2.imread(image_path)

# Perform inference
results = model(image)

# Parse and visualize results
annotated_frame = results[0].plot()

# Display the image with detections
cv2.imshow('YOLOv8n Detection', annotated_frame)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

# Optionally, save the annotated image
cv2.imwrite('annotated_image.jpg', annotated_frame)


In [ ]:
import cv2
from ultralytics import YOLO

# Load the YOLO models
vehicle_model = YOLO('yolov8n.pt')  # Vehicle detection model
plate_model = YOLO('license_plate_detector.pt')  # License plate detection model (change to your model path)

# Path to your image
image_path = "data\State-wise_OLX\AN\AN1.jpg"  # Replace with your image path

# Load the image using OpenCV
image = cv2.imread(image_path)

# Perform vehicle detection
vehicle_results = vehicle_model(image)

# Initialize lists for detected vehicles and their corresponding number plates
vehicles = []

# Loop through vehicle detection results
for result in vehicle_results:
    boxes = result.boxes
    for box in boxes:
        class_name = result.names[int(box.cls[0])]  # Get the class name
        if class_name in ['car', 'bus', 'truck', 'motorcycle']:  # Filter for vehicle classes
            vehicles.append(box.xyxy[0].cpu().numpy().astype(int))  # Store bounding box

# Now run license plate detection on the cropped vehicle images
for vehicle_box in vehicles:
    x_min, y_min, x_max, y_max = vehicle_box
    vehicle_crop = image[y_min:y_max, x_min:x_max]  # Crop the vehicle region

    # Perform license plate detection on the cropped vehicle image
    plate_results = plate_model(vehicle_crop)

    # Loop through plate detection results
    for plate_result in plate_results:
        plates = plate_result.boxes
        for plate in plates:
            plate_box = plate.xyxy[0].cpu().numpy().astype(int)
            # Annotate the license plate on the vehicle crop
            cv2.rectangle(vehicle_crop, (plate_box[0], plate_box[1]), (plate_box[2], plate_box[3]), (255, 0, 0), 2)  # Blue for plates

    # Display the annotated vehicle crop with plates
    cv2.imshow("Detected Vehicle with Plates", vehicle_crop)
    cv2.waitKey(0)

# Annotate and show the original image with vehicle detections
for vehicle_box in vehicles:
    cv2.rectangle(image, (vehicle_box[0], vehicle_box[1]), (vehicle_box[2], vehicle_box[3]), (0, 255, 0), 2)  # Green for vehicles

# Display the image with detections
cv2.imshow('YOLO Detection', image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

# Optionally, save the annotated image
cv2.imwrite('annotated_image.jpg', image)


In [ ]:
import cv2
from ultralytics import YOLO
import pytesseract

# Ensure to specify the path to tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Change to your Tesseract path

# Load the YOLOv8 Nano model for vehicle detection
model = YOLO('yolov8n.pt')

# Path to your image
image_path = "data\State-wise_OLX\AN\AN1.jpg"  # Replace with your image path

# Load the image using OpenCV
image = cv2.imread(image_path)

# Perform inference to detect vehicles
results = model(image)

# Initialize list to store vehicle bounding boxes
vehicles = []

# Loop through the detected objects to filter for vehicles
for result in results:
    boxes = result.boxes
    for box in boxes:
        # You can filter by class label, here we assume "vehicle" classes like car, truck, etc.
        class_name = result.names[int(box.cls[0])]  # Get the class name
        if class_name in ['car', 'bus', 'truck', 'motorcycle']:  # Filter vehicle classes
            vehicles.append(box.xyxy[0].cpu().numpy().astype(int))  # Store vehicle bounding box

# Now we process each detected vehicle
for vehicle_box in vehicles:
    x_min, y_min, x_max, y_max = vehicle_box
    vehicle_crop = image[y_min:y_max, x_min:x_max]  # Crop the vehicle region from the image
    
    # Display the detected vehicle
    cv2.imshow("Detected Vehicle", vehicle_crop)
    
    # Perform OCR to detect the number plate in the vehicle region
    # You can adjust the region where the number plate is likely to appear, based on your use case
    ocr_result = pytesseract.image_to_string(vehicle_crop)
    
    if ocr_result.strip():
        print("Detected Number Plate Text:", ocr_result)
    else:
        print("No number plate detected in this vehicle.")
    
    cv2.waitKey(0)

# Annotate and show the original image with vehicle detections
annotated_frame = results[0].plot()

# Display the image with detections
cv2.imshow('YOLOv8n Detection', annotated_frame)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

# Optionally, save the annotated image
cv2.imwrite('annotated_image.jpg', annotated_frame)


In [ ]:

import google.generativeai as genai
from IPython.display import Markdown
import PIL.Image

# Load API key from .env file
def load_api_key():
    with open('.env') as f:
        return dict(line.strip().split('=', 1) for line in f if '=' in line).get('gemini_api_key')
    
API_KEY = load_api_key()

genai.configure(api_key=API_KEY)

image_path = "data\State-wise_OLX\AN\AN1.jpg"

# Prompt
prompt = (
    "Identify if you could find a vehicle in the provided image. "
    "Classify if the vehicle is 'Car', 'Bike', 'Auto rickshaw', or 'Heavy Vehicle' and provide number plate details. "
    "Provide output in 'vehicle_type', 'vehicle_number', 'vehicle_approch[in-if the vechicle front side or out-if vechicle back side]'."
)

# Prepare input
myfile = genai.upload_file(path=image_path)
inputs = [prompt,myfile]

# Generate content
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(inputs)
myfile.delete()

# Display the response

response.resolve()
# Markdown("> " + response.text)
print(response.text)
